In [28]:
from sqlalchemy import create_engine, select, update
from sqlalchemy.orm import Session, joinedload
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import api.entities as db
from itemchm.entities_repr import User, Book

# Create engine
ENGINE = create_engine("sqlite:///../../bookshelf.db", echo=True)

In [31]:
def sentiment_analysis(text : str):
    # Create a SentimentIntensityAnalyzer object
    sia = SentimentIntensityAnalyzer()

    # Perform sentiment analysis
    sentiment_scores = sia.polarity_scores(text)
    
    if sentiment_scores['compound'] < -0.1:
        return 1
    elif sentiment_scores['compound'] < 0.1:
        return 4
    else:
        return 6
        
def overall_rate(user_book : db.UserBook) -> float:
    RATING_WEIGHT = 4
    COMMENT_WEIGHT = 3
    SHARED_WEIGHT = 2
    READ_RATIO_WEIGHT = 1

    if user_book.rating == None:
        rating_value = 4
    else:
        rating_value = user_book.rating + 1
    
    if user_book.comment == None:
        comment_value = 4
    else:
        comment_value = sentiment_analysis(user_book.comment)
    
    read_ration_value = user_book.readRatio * 5 + 1

    if user_book.shared <= 0:
        shared_value = 1
    elif user_book.shared < 3:
        shared_value = 2
    elif user_book.shared < 6:
        shared_value = 3
    elif user_book.shared < 8:
        shared_value = 4
    elif user_book.shared < 12:
        shared_value = 5
    else:
        shared_value = 6

    return (RATING_WEIGHT * rating_value + COMMENT_WEIGHT * comment_value + SHARED_WEIGHT * shared_value + READ_RATIO_WEIGHT * read_ration_value) / (RATING_WEIGHT + COMMENT_WEIGHT + SHARED_WEIGHT + READ_RATIO_WEIGHT)

In [30]:
with Session(ENGINE) as session:
    db_users = session.query(db.User).all()
    db_books = session.query(db.Book).options(joinedload(db.Book.genres)).all()
    db_user_books = session.query(db.UserBook).all()

2024-08-27 11:45:52,358 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-27 11:45:52,402 INFO sqlalchemy.engine.Engine SELECT "Users".id AS "Users_id", "Users".name AS "Users_name", "Users".email AS "Users_email", "Users".features AS "Users_features" 
FROM "Users"
2024-08-27 11:45:52,404 INFO sqlalchemy.engine.Engine [generated in 0.00377s] ()
2024-08-27 11:45:52,466 INFO sqlalchemy.engine.Engine SELECT "Books".id AS "Books_id", "Books".name AS "Books_name", "Books".author AS "Books_author", "Books".language AS "Books_language", "Books".year AS "Books_year", "BookGenre_1"."booksId" AS "BookGenre_1_booksId", "BookGenre_1"."genresName" AS "BookGenre_1_genresName" 
FROM "Books" LEFT OUTER JOIN "BookGenre" AS "BookGenre_1" ON "Books".id = "BookGenre_1"."booksId"
2024-08-27 11:45:52,467 INFO sqlalchemy.engine.Engine [generated in 0.00074s] ()
2024-08-27 11:45:53,831 INFO sqlalchemy.engine.Engine SELECT "UserBooks".id AS "UserBooks_id", "UserBooks"."readRatio" AS "UserBooks_readRatio",

In [33]:
users_dict : dict[int, User] = {}

for db_user in db_users:
    user = User(db_user.id, db_user.name, {})
    users_dict[db_user.id] = user

books : list[Book] = []

for db_book in db_books:
    genres = [book_genre.genresName for book_genre in db_book.genres]
    book = Book(db_book.id, db_book.name, db_book.author, db_book.year, db_book.language, genres)
    books.append(book)

In [35]:
len(db_user_books)

124738

In [34]:
for user_book in db_user_books:
    user = users_dict[user_book.userId]
    rating = overall_rate(user_book)
    user.ratings[user_book.bookId] = rating

In [37]:
users_dict[0].ratings

{67961: 4.82,
 13610: 1.725,
 22217: 1.225,
 1041: 1.405,
 25609: 1.3599999999999999,
 997: 1.7600000000000002,
 15: 2.17,
 2170: 2.2,
 54873: 2.2399999999999998,
 11: 1.645,
 2587: 2.145,
 1057: 1.105,
 52914: 1.8199999999999998,
 43851: 1.4,
 29433: 1.605,
 257: 1.2650000000000001,
 48438: 2.1350000000000002,
 39955: 2.045,
 1022: 1.475,
 35690: 1.83,
 14568: 1.225,
 19994: 2.1100000000000003,
 4039: 2.215,
 61: 1.4949999999999999,
 67138: 2.1149999999999998,
 10732: 1.97,
 10378: 2.0300000000000002,
 80: 1.045,
 33283: 1.41,
 2449: 2.0300000000000002,
 13: 2.085,
 31552: 1.975,
 41: 1.155,
 21279: 2.08,
 22984: 1.41,
 36034: 1.385,
 2147: 1.605,
 10607: 1.03,
 82: 1.81,
 8775: 2.035,
 30251: 2.055,
 284: 1.6800000000000002,
 4352: 1.03,
 15489: 1.3,
 1430: 1.7449999999999999,
 74076: 2.04,
 69700: 1.42,
 135: 1.435,
 1572: 4.4799999999999995,
 7524: 5.1899999999999995,
 26654: 2.025,
 56006: 2.0100000000000002,
 2048: 1.8199999999999998,
 22566: 1.2349999999999999,
 38956: 1.585,
 1